In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Install Required Packages

In [ ]:
%pip install --upgrade --quiet  langchain langchain-community langchain-ollama langchain-experimental neo4j tiktoken yfiles_jupyter_graphs python-dotenv json-repair langchain-openai langchain_core

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 78.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.7/301.7 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 61.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.3/454.3 kB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 83.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 

In [ ]:
!sudo apt-get install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh # download ollama api

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpci3 pci.ids
The following NEW packages will be installed:
  libpci3 pci.ids pciutils
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 343 kB of archives.
After this operation, 1,581 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 pci.ids all 0.0~2022.01.22-1 [251 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libpci3 amd64 1:3.7.0-6 [28.9 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 pciutils amd64 1:3.7.0-6 [63.6 kB]
Fetched 343 kB in 1s (318 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 3.)
debconf: falling back to frontend: Readline
debconf: unable to initializ

## Set Up Ollama API & Pull LLAMA 3.1 Model

In [ ]:
from IPython.display import clear_output
# Create a Python script to start the Ollama API server in a separate thread
import os
import threading
import subprocess
import requests
import json

def ollama():
    os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
    os.environ['OLLAMA_ORIGINS'] = '*'
    subprocess.Popen(["ollama", "serve"])

ollama_thread = threading.Thread(target=ollama)
ollama_thread.start()

In [ ]:
from IPython.display import clear_output

!ollama pull llama3.1
clear_output()

!pip install -U lightrag[ollama]
clear_output()

## Import modules

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_core.output_parsers import StrOutputParser
from langchain_community.graphs import Neo4jGraph
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.chat_models import ChatOllama  # Mise à jour de l'importation
from langchain_experimental.graph_transformers import LLMGraphTransformer
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget
from langchain_community.vectorstores import Neo4jVector
from langchain_community.document_loaders import PyPDFLoader  # Pour charger les fichiers PDF
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
from langchain_ollama import OllamaEmbeddings  # À garder si nécessaire

In [ ]:
import os
print(os.getcwd()) # prints your current working directory
!ls -l # lists content of your working directory, including files and their permissions.

/content
total 8
drwx------ 6 root root 4096 Dec 29 11:27 drive
drwxr-xr-x 1 root root 4096 Dec 19 14:20 sample_data


## Define Classes for Nodes, Relationships, and Documents

In [ ]:
from typing import List, Dict, Optional

class Node:
    def __init__(self, id: str, type: str, properties: Dict):
        self.id = id
        self.type = type
        self.properties = properties

    def __repr__(self):
        return f"Node(id='{self.id}', type='{self.type}', properties={self.properties})"

class Relationship:
    def __init__(self, source: Node, target: Node, type: str, properties: Dict):
        self.source = source
        self.target = target
        self.type = type
        self.properties = properties

    def __repr__(self):
        return f"Relationship(source={self.source}, target={self.target}, type='{self.type}', properties={self.properties})"

class Document:
    def __init__(self, metadata: Dict, page_content: str):
        self.metadata = metadata
        self.page_content = page_content

    def __repr__(self):
        return f"Document(metadata={self.metadata}, page_content='{self.page_content[:50]}...')"

class GraphDocument:
    def __init__(self, nodes: List[Node], relationships: List[Relationship], source: Document):
        self.nodes = nodes
        self.relationships = relationships
        self.source = source

    def __repr__(self):
        return f"GraphDocument(nodes={self.nodes}, relationships={self.relationships}, source={self.source})"


## Load extracted othologies file

In [ ]:
with open("progress.json", "r") as f:
    serialized_documents = json.load(f)

# Reconstitution des objets en format GraphDocument
GraphDocument
# Recréer les objets
graph_documents = [
    GraphDocument(**doc) if isinstance(doc, dict) else doc
    for doc in serialized_documents
]

In [ ]:
graph_documents[0]

GraphDocument(nodes=[], relationships=[], source={'id': None, 'metadata': {'source': 'data2.pdf', 'page': 0}, 'page_content': '1774 BULLETIN OFFICIEL Nº 7222 – 30 moharrem 1445 (17-8-2023) \n \n \n \nArrêté du ministre délégué auprès de la ministre de l’économie et des finances, chargé du budget n° 1692 -23 du 4 hija 1444  (23 juin 2023)', 'type': 'Document'})

## Load and Transform JSON Data to GraphDocuments

In [ ]:
import json


def transform_json_to_graph(json_data: List[Dict]) -> List[GraphDocument]:
    graph_documents = []

    for item in json_data:
        nodes = [Node(id=node['id'], type=node['type'], properties=node.get('properties', {}))
                 for node in item.get('nodes', [])]

        relationships = [
            Relationship(
                source=Node(id=rel['source']['id'], type=rel['source']['type'], properties=rel['source'].get('properties', {})),
                target=Node(id=rel['target']['id'], type=rel['target']['type'], properties=rel['target'].get('properties', {})),
                type=rel['type'],
                properties=rel.get('properties', {})
            ) for rel in item.get('relationships', [])
        ]

        # Vérifier si nodes ou relationships ne sont pas vides
        if not nodes and not relationships:
            continue  # Ignorer cet item

        source_document = Document(
            metadata=item['source']['metadata'],
            page_content=item['source']['page_content']
        )

        graph_documents.append(GraphDocument(nodes=nodes, relationships=relationships, source=source_document))

    return graph_documents

# Charger le fichier progress.json
with open("progress.json", "r", encoding="utf-8") as f:
    json_data = json.load(f)

# Transformer en GraphDocuments
graph_documents = transform_json_to_graph(json_data)

# Vérifier la sortie
print(f"Nombre de documents transformés : {len(graph_documents)}")
print("Exemple d'un document transformé :")
if graph_documents:
  graph_documents[0]


Nombre de documents transformés : 669
Exemple d'un document transformé :


In [ ]:
graph_documents[0]


GraphDocument(nodes=[Node(id='dahir n° 1 -07-129', type='Decree', properties={}), Node(id='loi n° 53 -05', type='Law', properties={})], relationships=[Relationship(source=Node(id='loi n° 53 -05', type='Law', properties={}), target=Node(id='dahir n° 1 -07-129', type='Decree', properties={}), type='RELATED_TO', properties={})], source=Document(metadata={'source': 'data2.pdf', 'page': 0}, page_content='Vu la loi n° 53 -05 relative à l’échange électroni...'))

## Set Up Neo4j Connection and Add GraphDocuments

In [ ]:
# os.environ["NEO4J_URI"] = ''
# os.environ["NEO4J_USERNAME"] = ""
# os.environ["NEO4J_PASSWORD"] = ""

#### First time

In [ ]:
from neo4j import GraphDatabase
from langchain.embeddings import OllamaEmbeddings
from langchain.graphs import Neo4jGraph
import os

# Set up Neo4j connection using environment variables
uri = os.environ.get("NEO4J_URI")  # Ensure these environment variables are set
user = os.environ.get("NEO4J_USERNAME")
password = os.environ.get("NEO4J_PASSWORD")

driver = GraphDatabase.driver(uri, auth=(user, password))

# Set up Neo4jGraph and embeddings
graph = Neo4jGraph(url=uri, username=user, password=password)

#
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

<ipython-input-15-2afd936a2a14>:14: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph(url=uri, username=user, password=password)


#### Second time

In [ ]:
# Set up Neo4j connection using environment variables
uri = os.environ.get("NEO4J_URI")  # Ensure these environment variables are set
user = os.environ.get("NEO4J_USERNAME")
password = os.environ.get("NEO4J_PASSWORD")

driver = GraphDatabase.driver(uri, auth=(user, password))

# Set up Neo4jGraph and embeddings
graph = Neo4jGraph(url=uri, username=user, password=password)

<ipython-input-13-266e6adf20dd>:9: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph(url=uri, username=user, password=password)


#### Embedding

In [ ]:
!ollama pull mxbai-embed-large
clear_output()

In [ ]:
# Assuming you have a valid embeddings model (e.g., OllamaEmbeddings)
embeddings = OllamaEmbeddings(model="mxbai-embed-large")

# Sample usage of graph (without Neo4jVector)
vector_index = Neo4jVector.from_existing_graph(
    embeddings,
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)
vector_retriever = vector_index.as_retriever()

#### Graph db Index

In [ ]:
driver = GraphDatabase.driver(
        uri = os.environ.get('NEO4J_URI'),
        auth = (os.environ.get('NEO4J_USERNAME'), os.environ.get('NEO4J_PASSWORD')))

def create_fulltext_index(tx):
    query = '''
    CREATE FULLTEXT INDEX `fulltext_entity_id`
    FOR (n:__Entity__)
    ON EACH [n.id];
    '''
    tx.run(query)

# Function to execute the query
def create_index():
    with driver.session() as session:
        session.execute_write(create_fulltext_index)
        print("Fulltext index created successfully.")

# Call the function to create the index
try:
    create_index()
except:
    pass

# Close the driver connection
driver.close()

#### Viz

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
def showGraph() :
    driver =GraphDatabase.driver(
    uri = os. environ ["NEO4J_URI" ],
    auth =(os.environ ["NEO4J_USERNAME" ],
    os. environ ["NEO4J_PASSWORD" ]))
    session = driver.session()
    widget = GraphWidget (graph = session.run("MATCH (s)-[r: !MENTIONS] ->(t) RETURN s,r,t") . graph())
    widget.node_label_mapping = 'id'
    return widget
showGraph()

GraphWidget(layout=Layout(height='800px', width='100%'))

Support for third party widgets will remain active for the duration of the session. To disable support:

In [ ]:
from google.colab import output
output.disable_custom_widget_manager()

## Define Functions for Entity Extraction and Graph Retrieval

In [ ]:
from pydantic import BaseModel, Field, ValidationError
import json


class Entities(BaseModel):
    """Identification des informations relatives aux entités."""
    names: list[str] = Field(
        ...,
        description="Toutes les entités de personnes, d'organisations ou d'entreprises qui apparaissent dans le texte.",
    )

# Placeholder function to remove special characters for Lucene-compatible queries
def remove_lucene_chars(input_text: str) -> str:
    # Define characters to remove as needed (e.g., special symbols)
    lucene_special_chars = "!@#$%^&*()[]{}|\\;:'\"<>,.?/"
    return ''.join(char for char in input_text if char not in lucene_special_chars)

# Generates a full-text query for Lucene-based search compatibility
def generate_full_text_query(input: str) -> str:
    words = [el for el in remove_lucene_chars(input).split() if el]
    if not words:
        return ""
    full_text_query = " AND ".join([f"{word}~2" for word in words])
    print(f"Generated Query: {full_text_query}")
    return full_text_query.strip()

# Extracts entities from text using the LLM
def extract_entities_from_text(question: str) -> Entities:
    # Format the prompt explicitly to ask for a JSON response
    formatted_prompt = (
        "You are extracting organization and person entities from the text. "
        "Extract entities from the following input and return them in JSON format. "
        "Use the format: {{'names': [...]}}. Input: {question}"
    ).format(question=question)

    # Ensure `llm` is initialized and invoke it with the prompt
    llm_response = llm.invoke(formatted_prompt)

    # Parse the response as JSON and validate it against the Entities model
    try:
        response_data = json.loads(llm_response.content)  # Parse the JSON response

        # Handle nested structure if 'entities' is the outer key
        if 'entities' in response_data and 'names' in response_data['entities']:
            response_data = response_data['entities']  # Extract only the inner 'names' dictionary

        entities = Entities.parse_obj(response_data)  # Validate and convert to Entities model
        print(f"Extracted Entities: {entities}")
        return entities
    except (json.JSONDecodeError, ValidationError) as e:
        print("Error parsing response:", e)
        return Entities(names=[])

# Full-text index query and graph retrieval function
def graph_retriever(question: str) -> str:
    """
Collecte le voisinage des entités mentionnées dans la question
    """
    result = ""
    # Step 1: Extract entities from the question using LLM
    entities = extract_entities_from_text(question)

    # Step 2: Retrieve graph data based on each entity
    for entity in entities.names:
        full_text_query = generate_full_text_query(entity)
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('fulltext_entity_id', $query, {limit:2})
            YIELD node, score
            CALL {
              WITH node
              MATCH (node)-[r:MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' + node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": full_text_query},
        )
        # Process and format the graph retrieval output
        result += "\n".join([el['output'] for el in response])
    return result

In [ ]:
llm = ChatOllama(model="llama3.1", temperature=0, format="json")  # Utilisation du modèle mis à jour

<ipython-input-20-0f415b4df970>:1: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="llama3.1", temperature=0, format="json")  # Utilisation du modèle mis à jour


In [ ]:
print(graph_retriever("Qui sont 'candidature' et 'architecte de l'administration' ?"))

<ipython-input-19-186de01d3bec>:47: PydanticDeprecatedSince20: The `parse_obj` method is deprecated; use `model_validate` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  entities = Entities.parse_obj(response_data)  # Validate and convert to Entities model


Extracted Entities: names=['Qui', "l'administration"]
Generated Query: Qui~2


Generated Query: ladministration~2


9d1b7b931b615e98e87a26b676fe7ec8 - MENTIONS -> qui leur sont reprochés
f5959a2847834e6308d9672feb6db367 - MENTIONS -> Les prestations qui peuvent faire l'objet de concours93d32a64fdd69d1172184c39d20520cb - MENTIONS -> architecte de l'administration
d35f8b31bdb3cd6345608012e314ac6c - MENTIONS -> administrations or competent bodies of their countries of origin or origin


## Define Full Retriever and Q&A Chain

In [ ]:
def full_retriever(question: str):
    graph_data = graph_retriever(question)
    vector_data = [el.page_content for el in vector_retriever.invoke(question)]

    final_data = f"""Graph data:
{graph_data}
vector data:
{"#Document ". join(vector_data)}
    """
    return final_data


In [ ]:
full_retriever("Qui sont 'candidature' et 'architecte de l'administration' ?")

<ipython-input-19-186de01d3bec>:47: PydanticDeprecatedSince20: The `parse_obj` method is deprecated; use `model_validate` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  entities = Entities.parse_obj(response_data)  # Validate and convert to Entities model


Extracted Entities: organizations=[] people=[] names=['Qui', "l'administration"]
Generated Query: Qui~2
Generated Query: ladministration~2


"Graph data:\n9d1b7b931b615e98e87a26b676fe7ec8 - MENTIONS -> qui leur sont reprochés\nf5959a2847834e6308d9672feb6db367 - MENTIONS -> Les prestations qui peuvent faire l'objet de concours93d32a64fdd69d1172184c39d20520cb - MENTIONS -> architecte de l'administration\nd35f8b31bdb3cd6345608012e314ac6c - MENTIONS -> administrations or competent bodies of their countries of origin or origin\nvector data:\n\ntext: b) Pour les préfectures ou provinces et leurs groupements : \n– le directeur général des services ou le directeur des services, selon le cas, ou son représentant  ; \n– le chef du service des marchés ou son représentant ;#Document \ntext: convoquer l’expert concerné pour participer aux travaux du  jury ou de la sous–commission.  \n3 – Tout architecte concurrent doit fournir, au moment  de sa candidature, un dossier administratif constitué de l’ensemble#Document \ntext: – de déchiffrer les candidatures et les offres électroniques déposées par les concurrents ; \n– d’ouvrir les plis de

In [ ]:
template = """Answer the question using the following context. Cite whether the information comes from 'Graph data' or 'Vector data':
{context}

Question: {question}
Use natural language, be concise, and provide accurate references.
Answer:"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
        {
            "context": full_retriever,
            "question": RunnablePassthrough(),
        }
    | prompt
    | llm
    | StrOutputParser()
)

### Q & A

In [ ]:
import logging

# Set up logging to suppress warnings from Neo4j
logging.getLogger("neo4j").setLevel(logging.ERROR)


In [ ]:
from pydantic import BaseModel, Field
from typing import Optional

class Entities(BaseModel):
    organizations: list = []
    people: list = []
    names: Optional[list] = []  # Rendre ce champ optionnel


In [ ]:
response_data = chain.invoke(input="Quelles sont les responsabilités des différents acteurs mentionnés dans cet arrêté, comme les maîtres d'ouvrage, les concurrents et les agents de contrôle ?")

# Récupérer uniquement la réponse générée, sans validation
# Exemple : Si la réponse est sous forme de dictionnaire avec une clé 'response'
print(response_data)


<ipython-input-19-186de01d3bec>:47: PydanticDeprecatedSince20: The `parse_obj` method is deprecated; use `model_validate` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  entities = Entities.parse_obj(response_data)  # Validate and convert to Entities model


Extracted Entities: organizations=[] people=[] names=[]
{ "Responsabilité du maître d'ouvrage" : 
-119.1  } 

  





  





  





  





  





  





  





  





  





  








In [ ]:
chain.invoke(input="Quelles sont les responsabilités des différents acteurs mentionnés dans cet arrêté, comme les maîtres d'ouvrage, les concurrents et les agents de contrôle ?")

<ipython-input-19-186de01d3bec>:47: PydanticDeprecatedSince20: The `parse_obj` method is deprecated; use `model_validate` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  entities = Entities.parse_obj(response_data)  # Validate and convert to Entities model


Extracted Entities: organizations=[] people=[] names=[]


'{ "Responsabilité du maître d\'ouvrage" : \n-119.1  } \n\n  \n\n\n\n\n\n  \n\n\n\n\n\n  \n\n\n\n\n\n  \n\n\n\n\n\n  \n\n\n\n\n\n  \n\n\n\n\n\n  \n\n\n\n\n\n  \n\n\n\n\n\n  \n\n\n\n\n\n  \n\n\n\n\n\n'

In [ ]:
import time
time.sleep(15 * 60)

### Gradio

In [ ]:
!pip install gradio

In [39]:
import gradio as gr

# Fonction pour obtenir la réponse à la question via votre pipeline existant
def get_answer(question: str):
    # Appeler votre chaîne de traitement (par exemple, chain.invoke())
    result = chain.invoke(input=question)

    # Si la sortie est un objet complexe, convertissez-la en chaîne de texte
    # Par exemple, si result est un dictionnaire, vous pouvez utiliser json.dumps pour le convertir en chaîne
    if isinstance(result, dict):
        result = json.dumps(result, indent=4)  # Formater en JSON lisible
    elif isinstance(result, str):
        pass  # La réponse est déjà une chaîne de caractères
    else:
        result = str(result)  # Convertir toute autre sortie en chaîne de caractères

    return result

# Créer l'interface Gradio
interface = gr.Interface(
    fn=get_answer,  # Fonction qui traite la question
    inputs="text",
    outputs="text",
    title="LLM Question-Answering Interface",
    description="Posez une question et recevez une réponse basée sur le contexte fourni."
)

# Lancer l'interface
interface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0e8c176617777edbff.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
